# Developing an analytical platform for evaluating the role of forest biorefineries in achieving a sustainable bioeconomy

The goal of this script is to optimize a biorefinery from environmental and economic perspectives, providing exact Pareto-optimal solutions for minimizing global warming potential (GWP) and maximizing net present value (NPV).

The optimization uses a multi-objective optimization (MOO) approach with non-dominated sorting to identify the best trade-offs between GWP and NPV. This approach ensures that the solutions provided come directly from the original dataset without modification or evolution, unlike genetic algorithms such as NSGA-II.


Key Variables:
--------------
- GWP per unit production of bioethanol, furfural, and vanillin.
- NPV for the three biochemicals.
- Conversion efficiencies (kg product/kg woodchips).
- Constraints ensure that the total woodchips used is exactly 100 kg and that all products must be produced in non-zero quantities.


Requirements:
-------------
1. `numpy`: Used for numerical operations, such as the handling of arrays.
   Install with: `pip install numpy==1.21.2`
      
2. `pandas`: Used for data manipulation and analysis, particularly for reading data from Excel files.
   Install with: `pip install pandas==1.3.3`
   
3. `matplotlib`: Used for visualizing the Pareto front of optimal solutions.
   Install with: `pip install matplotlib==3.4.3`

Steps in the Script:
--------------------
1. **Define the problem**: The biorefinery optimization problem is defined as minimizing GWP emissions and maximizing NPV. 
2. **Set constraints**: The total woodchips used must be 100 kg, and non-zero production of all three biochemicals is required.
3. **Non-dominated sorting**: The algorithm checks each point in the dataset and identifies whether it is dominated by any other point in terms of both GWP and NPV. Non-dominated solutions from the exact Pareto front.
4. **Visualize results**: The Pareto front is plotted to show the trade-offs between GWP and NPV.
5. **Display optimal production amounts**: The optimal production amounts for bioethanol, furfural, and vanillin are displayed for each solution on the Pareto front.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load your Excel sheet
file_path = r "./input/Curating_LCA_and_NPV.xlsx"  # Set the file path to your input Excel file (assumed to be in a local 'input' folder)

df = pd.read_excel(file_path)

In [ ]:
# Extract GWP and NPV from columns (GWP in 16th, NPV in 20th)
GWP = df.iloc[0:, 15].values  # Column 16 (zero-indexed)
NPV = df.iloc[0:, 19].values  # Column 20 (zero-indexed)

In [ ]:
bioethanol_percent = df.iloc[:, 0].values  
vanillin_percent = df.iloc[:, 1].values 
furfural_percent = df.iloc[:, 2].values 

In [ ]:
# Filter out solutions with negative NPV and prepare the filtered dataset
filtered_data = [(gwp, npv, bioethanol_percent[i], vanillin_percent[i], furfural_percent[i]) 
                 for i, (gwp, npv) in enumerate(zip(GWP, NPV)) if npv > 0]

In [ ]:
# Separate filtered GWP, NPV, and product percentages for use in Pareto sorting
GWP_filtered = [point[0] for point in filtered_data]
NPV_filtered = [point[1] for point in filtered_data]
bioethanol_filtered = [point[2] for point in filtered_data]
vanillin_filtered = [point[3] for point in filtered_data]
furfural_filtered = [point[4] for point in filtered_data]

In [ ]:
# Function to identify Pareto front
def is_dominated(point, others):
    """Check if a point is dominated by any in the list."""
    for other in others:
        if (other[0] <= point[0] and other[1] >= point[1]) and (other != point):
            return True
    return False

filtered_points = list(zip(GWP_filtered, NPV_filtered))
pareto_front = [point for point in filtered_points if not is_dominated(point, filtered_points)]

optimal_indices = [filtered_points.index(ind) for ind in pareto_front]


In [ ]:
# Extract Pareto-optimal GWP, NPV, and corresponding percentages for each biochemical
pareto_GWP = [point[0] for point in pareto_front]
pareto_NPV = [point[1] for point in pareto_front]
pareto_bioethanol = [bioethanol_filtered[idx] for idx in optimal_indices]
pareto_vanillin = [vanillin_filtered[idx] for idx in optimal_indices]
pareto_furfural = [furfural_filtered[idx] for idx in optimal_indices]

In [ ]:
# Plot the results
plt.scatter(GWP_filtered, NPV_filtered, label='All Solutions', color='blue')
plt.scatter(pareto_GWP, pareto_NPV, color='red', label='Pareto Front')
plt.xlabel('GWP')
plt.ylabel('NPV')
plt.title('Pareto-Optimal Solutions')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Create a DataFrame from the Pareto-optimal solutions with the respective percentages
pareto_df = pd.DataFrame({
    'Total GWP': pareto_GWP,
    'Total NPV': pareto_NPV,
    'Bioethanol %': pareto_bioethanol,
    'Vanillin %': pareto_vanillin,
    'Furfural %': pareto_furfural
})

In [ ]:
# Save the Pareto-optimal solutions along with the percentage breakdowns to an Excel file
output_file_path = 'Pareto Optimal Solutions V7.xlsx'
pareto_df.to_excel(output_file_path, index=False)

print(f"Pareto-optimal solutions with percentage breakdowns saved to {output_file_path}")

In [ ]:
# Plot the results with customized settings
fig, ax = plt.subplots(figsize=(8, 6))  # Adjust figure size as needed
fig.patch.set_facecolor('white')  # Set the face color to white

# Scatter plots for all solutions and Pareto front
ax.scatter(GWP_filtered, NPV_filtered, label='All Solutions', color='lightblue')
ax.scatter(pareto_GWP, pareto_NPV, color='green', label='Pareto Front')

# Set labels with custom font properties
ax.set_xlabel('GWP', fontname='Times New Roman', fontsize=12, fontweight='bold')
ax.set_ylabel('NPV', fontname='Times New Roman', fontsize=12, fontweight='bold')

# Move y-axis to the right
ax.yaxis.tick_right()                 # Move y-axis ticks to the right
ax.yaxis.set_label_position("right")   # Move y-axis label to the right

# Set title with custom font properties and padding
ax.set_title('Multi-objective Optimization', fontname='Times New Roman', fontsize=16, fontweight='bold', pad=40)

# Set x-axis and y-axis limits
ax.set_xlim(-13000, 0)
ax.set_ylim(0, 230000)

# Remove the left and top spines for aesthetics
ax.spines['left'].set_visible(False)
ax.spines['top'].set_visible(False)

# Modify legend to remove the box and set font size
ax.legend(frameon=False, fontsize=8)

# Show the plot
plt.show()


In [ ]:
min_gwp_index = np.argmin(pareto_GWP)  # Index of the point with the lowest GWP
print("Lowest GWP Configuration:")
print(pareto_df.iloc[min_gwp_index])  # Print the configuration details


In [ ]:
max_npv_index = np.argmax(pareto_NPV)  # Index of the point with the highest NPV
print("Highest NPV Configuration:")
print(pareto_df.iloc[max_npv_index])  # Print the configuration details


In [ ]:
from scipy.spatial.distance import euclidean

# Define an ideal point (lowest GWP & highest NPV)
ideal_gwp = min(pareto_GWP)
ideal_npv = max(pareto_NPV)

# Compute distances from ideal point
distances = [euclidean((gwp, npv), (ideal_gwp, ideal_npv)) for gwp, npv in zip(pareto_GWP, pareto_NPV)]

# Find the index of the knee point
knee_index = np.argmin(distances)

print("Balanced Trade-Off Configuration:")
print(pareto_df.iloc[knee_index])  # Print the configuration details
